# SST_grad preprocessing

This notebook shows how to generate the SST_grad_timeseries.nc file

In [1]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [2]:
from dask.distributed import Client

In [3]:
from utils import area,ccrs_land,add_patches
import datetime

In [4]:
c = Client()
c

Client Scheduler: tcp://127.0.0.1:40929 Dashboard: /proxy/8787/status,Cluster Workers: 6 Cores: 36 Memory: 154.62 GB


In [5]:
inputfiles = '/g/data/ua8/NOAA_OISST/AVHRR/v2-1_modified/oisst_avhrr_v2-1_*.nc'
# inputfiles='path2OISST/*.nc'

In [6]:
dataset_SST = xr.open_mfdataset(inputfiles,parallel=True)

In [7]:
dataset_SST = dataset_SST.sst.sel(time=slice('1992','2020')).squeeze().drop('zlev')

In [8]:
dataset_SST

<xarray.DataArray 'sst' (time: 10593, lat: 720, lon: 1440)>
dask.array<getitem, shape=(10593, 720, 1440), dtype=float32, chunksize=(366, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1992-01-01T12:00:00 ... 2020-12-31T12:00:00
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
Attributes:
    long_name:  Daily sea surface temperature
    units:      Celsius

Coarsen dataset to a 1 degree grid.

In [9]:
SST_coarsen = dataset_SST.coarsen({'lat':4,'lon':4}).mean().compute()

In [10]:
SST_coarsen_store = SST_coarsen.to_dataset(name='SST')

In [11]:
SST_coarsen_store.attrs['title'] = "Sea Surface Temperature (SST)"
SST_coarsen_store.attrs['Description'] = """SST from OISST-NOAA."""
SST_coarsen_store.attrs['Publication'] = "Coarsened data used in Martínez-Moreno, J. et. al. 2020: \n 'Global changes in oceanic mesoscale currents over the satellite altimetry record'"
SST_coarsen_store.attrs['Author'] = "Josué Martínez-Moreno"
SST_coarsen_store.attrs['Contact'] = "josue.martinezmoreno@anu.edu.au"

SST_coarsen_store.attrs['Created date'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

units = [r"$^\circ C$"]
names = ["SST"]
long_names = ["Sea Surface Temperature"]

counter=0
SST_coarsen_store['SST'].attrs['units'] = units[counter]
SST_coarsen_store['SST'].attrs['name'] = names[counter]
SST_coarsen_store['SST'].attrs['long_name'] = long_names[counter]

SST_coarsen_store['SST'].attrs['missing_value'] = np.nan
SST_coarsen_store['SST'].attrs['valid_min'] = np.nanmin(SST_coarsen_store['SST'])
SST_coarsen_store['SST'].attrs['valid_max'] = np.nanmax(SST_coarsen_store['SST'])
SST_coarsen_store['SST'].attrs['valid_range'] = [np.nanmin(SST_coarsen_store['SST']),np.nanmax(SST_coarsen_store['SST'])]

In [12]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in SST_coarsen_store.data_vars}

SST_coarsen_store.to_netcdf('../datasets/SST_timeseries.nc', encoding=encoding)

### Compute SST gradient magnitude:

\begin{equation}
|\nabla SST| = \sqrt{\left(\frac{\partial SST}{\partial x}\right)^2 + \left(\frac{\partial SST}{\partial y}\right)^2 }
\end{equation}

In [13]:
dy,dx = area(SST_coarsen.lat,SST_coarsen.lon,return_grid=True)

In [14]:
dSST_dx = SST_coarsen.differentiate('lon').squeeze() / dx
dSST_dy = SST_coarsen.differentiate('lat').squeeze() / dy[np.newaxis,:,np.newaxis]

In [15]:
SST_grad = np.sqrt(dSST_dx**2 + dSST_dy**2)

Rechunk dataset

In [16]:
SST_grad_rechunk = SST_grad.chunk({"lat": 40, "lon": 40,'time':355})

In [17]:
SST_grad_rechunk

,Array,Chunk
Bytes,2.75 GB,2.27 MB
Shape,"(10593, 180, 360)","(355, 40, 40)"
Count,1350 Tasks,1350 Chunks
Type,float32,numpy.ndarray


Roll over 12 months:

In [18]:
SST_grad_rolled = SST_grad_rechunk.rolling(time=365,center=True).mean().compute() #This computation requires ~ 50 GB of RAM.

In [19]:
SST_grad_rolled = SST_grad_rolled.to_dataset(name="SST")

In [20]:
SST_grad_rolled

<xarray.Dataset>
Dimensions:  (lat: 180, lon: 360, time: 10593)
Coordinates:
  * time     (time) datetime64[ns] 1992-01-01T12:00:00 ... 2020-12-31T12:00:00
  * lon      (lon) float32 0.5 1.5 2.5 3.5 4.5 ... 355.5 356.5 357.5 358.5 359.5
  * lat      (lat) float32 -89.5 -88.5 -87.5 -86.5 -85.5 ... 86.5 87.5 88.5 89.5
Data variables:
    SST      (time, lat, lon) float32 nan nan nan nan nan ... nan nan nan nan

### Add metadata

In [21]:
SST_grad_rolled.attrs['title'] = "Sea Surface Temperature (SST) gradients"
SST_grad_rolled.attrs['Description'] = """SST gradients computed from OISST-NOAA."""
SST_grad_rolled.attrs['Publication'] = "Dataset created for Martínez-Moreno, J. et. al. 2020: \n 'Global changes in oceanic mesoscale currents over the satellite altimetry record'"
SST_grad_rolled.attrs['Author'] = "Josué Martínez-Moreno"
SST_grad_rolled.attrs['Contact'] = "josue.martinezmoreno@anu.edu.au"

SST_grad_rolled.attrs['Created date'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

units = [r"$^\circ C m^{-1}$"]
names = ["SST_grad"]
long_names = ["Sea Surface Temperature gradient magnitude"]

counter=0
    
SST_grad_rolled["SST"].attrs['units'] = units[counter]
SST_grad_rolled["SST"].attrs['name'] = names[counter]
SST_grad_rolled["SST"].attrs['long_name'] = long_names[counter]

SST_grad_rolled["SST"].attrs['missing_value'] = np.nan
SST_grad_rolled["SST"].attrs['valid_min'] = np.nanmin(SST_grad_rolled["SST"])
SST_grad_rolled["SST"].attrs['valid_max'] = np.nanmax(SST_grad_rolled["SST"])
SST_grad_rolled["SST"].attrs['valid_range'] = [np.nanmin(SST_grad_rolled["SST"]),np.nanmax(SST_grad_rolled["SST"])]


## Store netCDF

In [22]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in SST_grad_rolled.data_vars}

SST_grad_rolled.to_netcdf('../datasets/SST_grad_timeseries.nc', encoding=encoding)